In [44]:
import sys
sys.path.append('../..')
from sqlalchemy import create_engine, MetaData, select, Table
import schedule
import pandas as pd
from datetime import datetime
import time
from config import POSTGRES_UTEA

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

metadata = MetaData()
reporte_tbl = Table("reporte", metadata, autoload_with=ENGINE, schema="datos_iag")

In [41]:
def get_datos():
    try:
        with ENGINE.begin() as conn:
            stmt = select(reporte_tbl).where(reporte_tbl.c.id == 1)
            result = conn.execute(stmt).fetchone()
            if result:
                return dict(result._mapping)  # convierte Row en dict
            else:
                print("⚠️ No se encontró el registro con id = 1")
                return None
    except Exception as e:
        print("❌ Error al hacer SELECT del reporte:", e)
        return None
        
def crear_mensaje(datos):
    mensaje = f'''*REPORTE*
*🕰️ Hora act.:* {datos['hora']}
*⚙️ Trapiches:* {datos['trapiches_estado']}
*🚛 Viajes disponibles:* {round(datos['viajes_disponibles'],2)}
*🔢 Toneladas aprox.:* {round(datos['toneladas_aprox'],2)}
*⏱️ Promedio llegada vj.:* {round(datos['promedio_llegada_viajes'],2)}
*📈 Viajes estimados:* {round(datos['viajes_estimados'],2)}
*🕰️ Total horas abas.:* {round(datos['total_horas_abastecimiento'],2)}
*⏳ Tiempo espera:* {round(datos['tiempo_espera'],2)}
*🎋 Molienda actual:* {round(datos['molienda_actual'],2)}
*📅 Planificacion actual:* {round(datos['planificacion_actual'],2)}
*🔻 Diferencia actual:* {round(datos['diferencia_actual'],2)}
*🕒 Promedio horario:* {round(datos['promedio_horario'],2)}
*🏭 Molienda segun promedio:* {round(datos['molienda_segun_promedio'],2)}
*📊 Molienda segun estimacion:* {round(datos['molienda_segun_estimacion'],2)}'''

    mario_sanchez = '75380725'
    harold_pincker = '70249286'
    me = '78194371'

    data_mensajes = {
        "fecha_registro": [datetime.now(), datetime.now(), datetime.now()],
        "cod_canero": [0, 0, 0],
        "nombre_canero": ["Mario Sanchez Romero", "Harold Pinker", "Bismark Socompi"],
        "numero_cel": [mario_sanchez, harold_pincker, me],
        "mensaje": [mensaje, mensaje, mensaje],
        "enviado": [False, False, False],
        "fecha_enviado": [None, None, None]
    }
    
    df = pd.DataFrame(data_mensajes)
    
    try:
        with ENGINE.begin() as conn:
            conn.execute(msj_whatsapp_tbl.insert(), df.to_dict(orient='records'))
        print("✅ Se ha actualizado ")
    except Exception as e:
        print("❌ Error al insertar en tabla MSJ WHASTAPP", e)
    #return df

def enviar_reporte_genencia():
    datos = get_datos()
    crear_mensaje(datos)

In [42]:
schedule.every().day.at("00:00").do(enviar_reporte_genencia)
schedule.every().day.at("03:00").do(enviar_reporte_genencia)
schedule.every().day.at("06:00").do(enviar_reporte_genencia)
schedule.every().day.at("09:00").do(enviar_reporte_genencia)
schedule.every().day.at("12:00").do(enviar_reporte_genencia)
schedule.every().day.at("15:00").do(enviar_reporte_genencia)
schedule.every().day.at("18:00").do(enviar_reporte_genencia)
schedule.every().day.at("21:00").do(enviar_reporte_genencia)

Every 1 day at 21:00:00 do enviar_reporte_genencia() (last run: [never], next run: 2025-06-22 21:00:00)

In [ ]:
enviar_reporte_genencia()
while True:
    schedule.run_pending()
    time.sleep(1)

✅ Se ha actualizado 
